## **Parameters/inputs/imports**

#### **Imports**

In [1]:
import pandas as pd

from ipywidgets import interact, fixed, interactive
import ipywidgets as widgets

from stadincijfers import stadincijfers

from settings import UPLOAD_DIR
from util.misc import _read_excel, _read_excel_lower
from util.parse_df import Parse_df
from util.conn_pinc_data import Conn_pinc_data

#### **Parameters**

In [2]:
all_pinc_periods = True

#### **PinC upload file**

In [3]:
filename = '20210623_Upload crimi en verkeer.xlsx'
#filename = 'Output_GIS_jaargang2013.xlsx'
file = UPLOAD_DIR + '/' + filename
print(file)

./upload_pinc/20210623_Upload crimi en verkeer.xlsx


#### **Import PinC upload file for validation**

In [4]:
data_table = _read_excel_lower(file,sheet_name=0)

In [5]:
data_table.head(2)

,period,geolevel,geoitem,v2802_vh_ver_do_aant,v2802_vh_ver_zgw_aant,v2802_vh_ver_lgw_aant
0,2019,gemeente,11001,3,8,87
1,2019,gemeente,11002,12,176,2369


#### **Parse data_table**

In [6]:
parsed_df = Parse_df(data_table)

In [7]:
period_list_pinc_query = parsed_df.determine_years(all_pinc_periods, _ind_id = 0)
print(f'Period list for Pinc Query: {period_list_pinc_query} \n')
var_list_pinc_query = parsed_df.determine_indicators()
print(f'Indicator list for PinC Query: {var_list_pinc_query} \n')
geolevel = parsed_df.determine_geolevel()
print(f'Geolevel for PinC Query: {geolevel}')

List of upload table years: ['2019', '2020']
Period list for Pinc Query: 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020 

Indicator list for PinC Query: v2802_vh_ver_do_aant,v2802_vh_ver_zgw_aant,v2802_vh_ver_lgw_aant 

Geolevel for PinC Query: gemeente


#### **PinC Query**

In [8]:
# Instantiate object from class
pinc = stadincijfers("provincies")

In [9]:
pinc_table = pinc.selectiontableasDataframe(var_list_pinc_query, geolevel=geolevel, periodlevel='year',period=period_list_pinc_query)

In [10]:
pinc_table.head(2)

,Geo,Perioden,doden na 30 dagen,zwaargewonden,lichtgewonden
0,Aalst,2009,6.0,28,403
1,Aalst,2010,2.0,33,396


In [11]:
if pinc_table.columns[0] != 'Geo':
    pinc_table.rename(columns={pinc_table.columns[0]: 'Geo'}, inplace = True)

In [12]:
geolevel

'gemeente'

## ***Interactive check framework:***

In [13]:
connection = Conn_pinc_data(pinc_table,data_table, geolevel)

#### **Plot**

In [14]:
list_vars = sorted(list(connection.cols_to_dict().keys()))
list_geos = sorted(list(connection.reversed_level_code_dict().keys())) 

In [15]:
interact(connection.draw_figure,var=list_vars,geo=list_geos, constant=fixed(0));

interactive(children=(Dropdown(description='var', options=('doden na 30 dagen', 'lichtgewonden', 'zwaargewonde…

#### **Overall outlier analysis**

In [16]:
interact(connection.overall_outlier_analysis, pinc_year=parsed_df.pinc_q_years, upload_year = parsed_df.years, absolute = True, ignore_inf = True);

interactive(children=(Dropdown(description='pinc_year', options=('2009', '2010', '2011', '2012', '2013', '2014…

#### **Univariate outlier analysis:**

In [17]:
interact(connection.show_outliers, var=list_vars,upload_year = parsed_df.years, pinc_year = parsed_df.pinc_q_years);

interactive(children=(Dropdown(description='var', options=('doden na 30 dagen', 'lichtgewonden', 'zwaargewonde…